# 아티스트 정보 크롤링 및 이미지 다운로드

이 노트북은 Artist Company 웹사이트에서 아티스트 정보와 이미지를 크롤링합니다.

## 주요 기능
- 아티스트 목록 추출
- 이미지 다운로드
- CSV 파일로 데이터 저장

## 성능 최적화
- **Selenium WebDriver**: 동적 콘텐츠 로딩 지원
- **Headless 모드**: 브라우저 UI 없이 실행
- **명시적 대기(WebDriverWait)**: 요소 로딩 대기로 안정성 향상
- **lxml 파서**: html.parser 대비 빠른 파싱
- **SoupStrainer**: 선택적 파싱으로 메모리 절약
- **requests.Session**: connection pooling으로 이미지 다운로드 최적화
- **함수 모듈화**: 재사용성 및 유지보수성 향상

In [119]:
import requests
from bs4 import BeautifulSoup, SoupStrainer
import os
from urllib.parse import urljoin, urlparse
import pandas as pd
from typing import List, Tuple, Optional

# Selenium imports
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException

In [120]:
# 설정 상수
BASE_URL = "https://www.artistcompany.co.kr/artist/"
CONTENT_DIR = "content"
TIMEOUT = 10
CHUNK_SIZE = 8192  # 이미지 다운로드 청크 크기
WAIT_TIMEOUT = 15  # Selenium 대기 시간 (초)

# User-Agent 및 기타 헤더 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

# Selenium 설정
USE_HEADLESS = True  # Headless 모드 사용 여부

In [121]:
def create_driver(headless: bool = True) -> webdriver.Chrome:
    """
    Chrome WebDriver를 생성하고 최적화된 옵션을 설정합니다.
    
    Args:
        headless: Headless 모드 사용 여부
        
    Returns:
        Chrome WebDriver 인스턴스
    """
    chrome_options = Options()
    
    # Headless 모드 설정
    if headless:
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--headless=new')  # 새로운 headless 모드
    
    # 성능 최적화 옵션
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument('--disable-extensions')
    chrome_options.add_argument('--disable-images')  # 이미지 로딩 비활성화로 속도 향상
    
    # User-Agent 설정
    chrome_options.add_argument(f'--user-agent={HEADERS["User-Agent"]}')
    
    # 로그 레벨 설정
    chrome_options.add_argument('--log-level=3')  # INFO 레벨 이상만 로그
    
    # 자동화 감지 방지
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    
    try:
        driver = webdriver.Chrome(options=chrome_options)
        # 페이지 로드 전략 설정 (필요한 리소스만 로드)
        driver.set_page_load_timeout(WAIT_TIMEOUT)
        return driver
    except WebDriverException as e:
        print(f"[오류] WebDriver 생성 실패: {e}")
        print("[정보] ChromeDriver가 설치되어 있는지 확인하세요.")
        raise

def fetch_html_with_selenium(url: str, wait_timeout: int = WAIT_TIMEOUT) -> BeautifulSoup:
    """
    Selenium을 사용하여 웹페이지를 가져와서 BeautifulSoup 객체로 반환합니다.
    동적 콘텐츠가 로드될 때까지 대기합니다.
    
    Args:
        url: 크롤링할 URL
        wait_timeout: 대기 시간 (초)
        
    Returns:
        BeautifulSoup 객체
        
    Raises:
        Exception: WebDriver 오류 또는 페이지 로드 실패 시
    """
    driver = None
    try:
        print(f"[정보] Selenium WebDriver 초기화 중...")
        driver = create_driver(headless=USE_HEADLESS)
        
        print(f"[정보] {url} 페이지 로딩 중...")
        driver.get(url)
        
        # 페이지가 완전히 로드될 때까지 대기
        # 'artist_sort' 클래스를 가진 요소가 나타날 때까지 대기
        wait = WebDriverWait(driver, wait_timeout)
        try:
            # 아티스트 항목이 로드될 때까지 대기
            wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.artist_sort, div.gallery, div.content-area"))
            )
            print("[정보] 페이지 요소 로딩 완료")
        except TimeoutException:
            print("[경고] 타임아웃: 일부 요소가 로드되지 않았을 수 있습니다.")
        
        # 추가 대기 (JavaScript 실행 완료를 위해)
        import time
        time.sleep(2)
        
        # 페이지 소스 가져오기
        page_source = driver.page_source
        
        # BeautifulSoup으로 파싱
        # lxml 파서 사용 (없으면 html.parser로 fallback)
        try:
            strainer = SoupStrainer("div")
            bs_obj = BeautifulSoup(page_source, "lxml", parse_only=strainer)
        except Exception:
            # lxml이 없으면 html.parser 사용
            print("[정보] lxml 파서를 사용할 수 없어 html.parser를 사용합니다.")
            strainer = SoupStrainer("div")
            bs_obj = BeautifulSoup(page_source, "html.parser", parse_only=strainer)
        
        return bs_obj
        
    finally:
        if driver:
            driver.quit()
            print("[정보] WebDriver 종료")

# HTML 가져오기 (Selenium 사용)
print(f"[정보] {BASE_URL} 페이지를 가져오는 중...")
bs_obj = fetch_html_with_selenium(BASE_URL)
print("[완료] HTML 파싱 완료")

[정보] https://www.artistcompany.co.kr/artist/ 페이지를 가져오는 중...
[정보] Selenium WebDriver 초기화 중...
[정보] https://www.artistcompany.co.kr/artist/ 페이지 로딩 중...
[정보] 페이지 요소 로딩 완료
[정보] lxml 파서를 사용할 수 없어 html.parser를 사용합니다.
[정보] WebDriver 종료
[완료] HTML 파싱 완료


In [122]:
def find_artist_items(bs_obj: BeautifulSoup) -> List:
    """
    HTML에서 아티스트 관련 div 요소를 효율적으로 찾습니다.
    출력 결과를 바탕으로 'artist_sort' 클래스를 가진 요소를 우선 탐색합니다.
    
    Args:
        bs_obj: BeautifulSoup 객체
        
    Returns:
        아티스트 관련 div 요소 리스트
    """
    artist_items = []
    
    # 1. 'artist_sort' 클래스를 가진 div 직접 탐색 (가장 효율적)
    artist_items = bs_obj.find_all("div", class_=lambda x: x and "artist_sort" in x)
    
    # 2. gallery 영역 탐색 (fallback)
    if not artist_items:
        gallery = bs_obj.find("div", class_="gallery")
        if gallery:
            artist_items = gallery.find_all("div", class_=lambda x: x and "artist" in str(x))
    
    # 3. content-area 내부 탐색 (fallback)
    if not artist_items:
        content_area = bs_obj.find("div", class_="content-area")
        if content_area:
            artist_items = content_area.find_all("div", class_=lambda x: x and "artist" in str(x))
    
    return artist_items

# 아티스트 항목 찾기
artist_items = find_artist_items(bs_obj)
print(f"[확인] 추출된 artist 관련 div 개수: {len(artist_items)}")

if not artist_items:
    print("[경고] artist 관련 div를 찾지 못했습니다. HTML 구조를 확인하세요.")

[확인] 추출된 artist 관련 div 개수: 23


In [123]:
import re

def extract_korean_name(text: str) -> Optional[str]:
    """
    텍스트에서 한글 이름만 추출합니다.
    
    Args:
        text: 전체 이름 텍스트 (예: "안성기Ahn Sung ki")
        
    Returns:
        한글 이름만 추출된 문자열 (예: "안성기") 또는 None
    """
    # 한글만 추출 (가-힣, 공백 포함)
    korean_match = re.search(r'^([가-힣\s]+)', text)
    if korean_match:
        korean_name = korean_match.group(1).strip()
        if korean_name:
            return korean_name
    return None

In [124]:
def extract_image_url(item, base_url: str) -> Optional[str]:
    """
    div 요소에서 이미지 URL을 추출합니다.
    
    Args:
        item: BeautifulSoup 요소
        base_url: 기본 URL (상대 경로 변환용)
        
    Returns:
        이미지 URL 또는 None
    """
    img_tag = item.find("img")
    if not img_tag:
        return None
    
    # src, data-src, srcset 순서로 시도
    img_src = (
        img_tag.get("src") or 
        img_tag.get("data-src") or 
        (img_tag.get("srcset") and img_tag.get("srcset").split()[0]) or 
        ""
    )
    
    if not img_src or "base64" in img_src:
        return None
    
    # 상대 경로 처리
    if img_src.startswith("//"):
        img_src = "https:" + img_src
    
    return urljoin(base_url, img_src)

def extract_artist_name(item) -> Optional[str]:
    """
    div 요소에서 아티스트 이름을 추출하고 한글 이름만 반환합니다.
    
    Args:
        item: BeautifulSoup 요소
        
    Returns:
        한글 이름만 추출된 문자열 (예: "안성기") 또는 None
    """
    # 1. 'name' 클래스를 가진 태그 찾기
    name_tag = item.find(lambda tag: tag.name in ["span", "p", "div"] 
                         and tag.get("class") and "name" in str(tag.get("class")))
    if name_tag:
        artist_name = name_tag.get_text(strip=True)
        if artist_name:
            # 한글 이름만 추출
            korean_name = extract_korean_name(artist_name)
            if korean_name:
                return korean_name
    
    # 2. 한글이 포함된 텍스트 찾기 (2-20자)
    for tag in item.find_all(["span", "p", "div"]):
        text = tag.get_text(strip=True)
        if 2 <= len(text) <= 20 and any('\uac00' <= ch <= '\ud7af' for ch in text):
            # 한글 이름만 추출
            korean_name = extract_korean_name(text)
            if korean_name:
                return korean_name
    return None

In [125]:
def download_image(session: requests.Session, img_url: str, img_path: str) -> bool:
    """
    이미지를 다운로드합니다. Session을 사용하여 connection pooling을 활용합니다.
    
    Args:
        session: requests.Session 객체
        img_url: 이미지 URL
        img_path: 저장할 파일 경로
        
    Returns:
        다운로드 성공 여부
    """
    try:
        with session.get(img_url, stream=True, timeout=TIMEOUT) as response:
            response.raise_for_status()
            with open(img_path, "wb") as f:
                for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
                    if chunk:
                        f.write(chunk)
            return True
    except requests.exceptions.RequestException as e:
        print(f"[오류] 이미지 다운로드 실패: {img_url} -> {e}")
        return False
    except Exception as e:
        print(f"[오류] 예상치 못한 오류: {img_url} -> {e}")
        return False

# 디렉터리 생성
os.makedirs(CONTENT_DIR, exist_ok=True)
print(f"[정보] 저장 디렉터리: {CONTENT_DIR}")

[정보] 저장 디렉터리: content


In [126]:
def crawl_artists(artist_items: List, base_url: str, content_dir: str, headers: dict) -> List[Tuple[str, str, str]]:
    """
    아티스트 정보를 크롤링하고 이미지를 다운로드합니다.
    
    Args:
        artist_items: 아티스트 div 요소 리스트
        base_url: 기본 URL
        content_dir: 이미지 저장 디렉터리
        headers: HTTP 헤더
        
    Returns:
        (아티스트명, 파일명, 이미지URL) 튜플 리스트
    """
    artist_data = []
    
    # Session을 사용하여 connection pooling 활용
    with requests.Session() as session:
        session.headers.update(headers)
        
        for idx, item in enumerate(artist_items, 1):
            # 이미지 URL 추출
            img_url = extract_image_url(item, base_url)
            if not img_url:
                print(f"[{idx}/{len(artist_items)}] 이미지 URL을 찾지 못했습니다. (건너뜀)")
                continue
            
            # 파일명 추출 및 검증
            img_name = os.path.basename(urlparse(img_url).path)
            if not img_name or len(img_name) < 3:
                print(f"[{idx}/{len(artist_items)}] 잘못된 이미지 이름: {img_name} (건너뜀)")
                continue
            
            # 아티스트 이름 추출
            artist_name = extract_artist_name(item)
            if not artist_name:
                print(f"[{idx}/{len(artist_items)}] 아티스트 이름을 찾지 못했습니다. (건너뜀)")
                continue
            
            # 이미지 다운로드 (이미 존재하면 건너뜀)
            img_save_path = os.path.join(content_dir, img_name)
            if not os.path.exists(img_save_path):
                print(f"[{idx}/{len(artist_items)}] 다운로드 중: {artist_name} -> {img_name}")
                if not download_image(session, img_url, img_save_path):
                    continue
            else:
                print(f"[{idx}/{len(artist_items)}] 이미 존재: {img_name} (건너뜀)")
            
            artist_data.append((artist_name, img_name, img_url))
    
    return artist_data

# 아티스트 정보 크롤링 실행
print(f"\n[시작] {len(artist_items)}개 항목 크롤링 시작...")
artist_data = crawl_artists(artist_items, BASE_URL, CONTENT_DIR, HEADERS)
print(f"\n[완료] {len(artist_data)}개 아티스트 정보 수집 완료")


[시작] 23개 항목 크롤링 시작...
[1/23] 다운로드 중: 안성기Ahn Sung ki -> ahn_sung_ki_list.png
[2/23] 다운로드 중: 이정재Lee Jung Jae -> portfolio_img_thumnail06.jpg
[3/23] 다운로드 중: 정우성Jung Woo Sung -> artist-03.jpg
[4/23] 다운로드 중: 염정아Yum Jung Ah -> yumjungah_profile.jpg
[5/23] 다운로드 중: 박해진Park Hae Jin -> 아티스트컴퍼니_artist_main_박해진.jpg
[6/23] 다운로드 중: 김종수Kim Jong Soo -> artist26-1-1-1.png
[7/23] 다운로드 중: 박소담Park So Dam -> 박소담_thum-1.jpg
[8/23] 다운로드 중: 배성우Bae Seong Woo -> baeseongwoo_profile.jpg
[9/23] 다운로드 중: 임지연Im Ji Yeon -> 아티스트컴퍼니_artist_main-1.jpg
[10/23] 다운로드 중: 신정근Shin Jung Keun -> artist16.jpg
[11/23] 다운로드 중: 김준한Kim Jun Han -> 아티스트컴퍼니_artist_main.jpg
[12/23] 다운로드 중: 박 훈Park Hoon -> 박훈_프로필_thum.jpg
[13/23] 다운로드 중: 원진아Won Jin A -> 원진아_프로필_thum-1.jpg
[14/23] 다운로드 중: 고아성Ko A Seong -> 고아성_프로필_thum-1.jpg
[15/23] 아티스트 이름을 찾지 못했습니다. (건너뜀)
[16/23] 다운로드 중: 이주영Lee Joo Young -> artist_main-1.jpg
[17/23] 다운로드 중: 김혜윤Kim Hye Yoon -> 김혜윤_thum.jpg
[18/23] 다운로드 중: 조이현Cho Yi Hyun -> 조이현-메인.jpg
[19/23] 다운로드 중: 장동주Jang Dong Ju -> PC

In [127]:
def save_to_csv(artist_data: List[Tuple[str, str, str]], csv_path: str):
    """
    아티스트 데이터를 CSV 파일로 저장합니다.
    pandas를 사용하여 더 안전하고 효율적으로 저장합니다.
    
    Args:
        artist_data: (아티스트명, 파일명, 이미지URL) 튜플 리스트
        csv_path: 저장할 CSV 파일 경로
    """
    if not artist_data:
        print("[경고] 저장할 데이터가 없습니다.")
        return
    
    df = pd.DataFrame(artist_data, columns=["artist_name", "file_name", "img_url"])
    df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    print(f"[저장 완료] CSV 파일: {csv_path}")
    print(f"[요약] 총 {len(df)}개 아티스트 정보 저장됨")
    print("\n[데이터 미리보기]")
    print(df.head(10).to_string(index=False))

# 결과 저장
csv_path = os.path.join(CONTENT_DIR, "artist_images.csv")
save_to_csv(artist_data, csv_path)



[저장 완료] CSV 파일: content/artist_images.csv
[요약] 총 22개 아티스트 정보 저장됨

[데이터 미리보기]
      artist_name                    file_name                                                                                 img_url
   안성기Ahn Sung ki         ahn_sung_ki_list.png         https://www.artistcompany.co.kr/wp-content/uploads/2021/08/ahn_sung_ki_list.png
  이정재Lee Jung Jae portfolio_img_thumnail06.jpg https://www.artistcompany.co.kr/wp-content/uploads/2016/11/portfolio_img_thumnail06.jpg
 정우성Jung Woo Sung                artist-03.jpg                https://www.artistcompany.co.kr/wp-content/uploads/2016/11/artist-03.jpg
   염정아Yum Jung Ah        yumjungah_profile.jpg        https://www.artistcompany.co.kr/wp-content/uploads/2017/03/yumjungah_profile.jpg
  박해진Park Hae Jin  아티스트컴퍼니_artist_main_박해진.jpg  https://www.artistcompany.co.kr/wp-content/uploads/2022/07/아티스트컴퍼니_artist_main_박해진.jpg
  김종수Kim Jong Soo           artist26-1-1-1.png           https://www.artistcompany.co.kr/wp-content/uploads/2017/

In [128]:
# 데이터프레임 생성
df_artists = pd.DataFrame(artist_data, columns=["artist_name", "file_name", "img_url"])
df_artists

,artist_name,file_name,img_url
0,안성기Ahn Sung ki,ahn_sung_ki_list.png,https://www.artistcompany.co.kr/wp-content/upl...
1,이정재Lee Jung Jae,portfolio_img_thumnail06.jpg,https://www.artistcompany.co.kr/wp-content/upl...
2,정우성Jung Woo Sung,artist-03.jpg,https://www.artistcompany.co.kr/wp-content/upl...
3,염정아Yum Jung Ah,yumjungah_profile.jpg,https://www.artistcompany.co.kr/wp-content/upl...
4,박해진Park Hae Jin,아티스트컴퍼니_artist_main_박해진.jpg,https://www.artistcompany.co.kr/wp-content/upl...
5,김종수Kim Jong Soo,artist26-1-1-1.png,https://www.artistcompany.co.kr/wp-content/upl...
6,박소담Park So Dam,박소담_thum-1.jpg,https://www.artistcompany.co.kr/wp-content/upl...
7,배성우Bae Seong Woo,baeseongwoo_profile.jpg,https://www.artistcompany.co.kr/wp-content/upl...
8,임지연Im Ji Yeon,아티스트컴퍼니_artist_main-1.jpg,https://www.artistcompany.co.kr/wp-content/upl...
9,신정근Shin Jung Keun,artist16.jpg,https://www.artistcompany.co.kr/wp-content/upl...
